In [3]:
# Librerias necesarias
import os
import sys
import json
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from generacion_pacientes import generar_pacientes
from collections import deque
# Add the parent directory to sys.path to allow relative imports
sys.path.append(os.path.abspath(os.path.join('..', '1. codigo analisis')))
import parametros as p

In [4]:
# Cargo los datos necesarios y transformo las llaves nuevamente a int (que se habian vuelto str)
incertidumbre = {}
for hospital in range(0,4): # 0 son llegadas a WL
    incertidumbre[hospital] = {}
    for requerimiento in range(1,4):
        incertidumbre[hospital][requerimiento] = {}

with open("resultados incertidumbre/incertidumbre_simulada.json", "r") as file:
    incertidumbre_keys_str = json.load(file)

for hospital in range(0,4): # 0 son llegadas a WL
    for requerimiento in range(1,4):
        for grd in range(1,9):
            incertidumbre[hospital][requerimiento][grd] = []
            lista = incertidumbre_keys_str[str(hospital)][str(requerimiento)][str(grd)]
            if lista != []:
                for data in lista:
                    arreglado = {
                    'TI': data["TI"],
                    'camino': {1: data['camino']["1"], 2: data['camino']["2"], 3: data['camino']["3"]},
                    'espera': {1: data['espera']["1"], 2: data['espera']["2"], 3: data['espera']["3"]}
                    }
                    incertidumbre[hospital][requerimiento][grd].append(arreglado)
            else:
                incertidumbre[hospital][requerimiento][grd] = []


In [5]:
class Paciente:
    CONTADOR_ID = 1  # Variable de clase para contar el número de Pacientes creados

    def __init__(self, hospital_llegada, requerimiento_inicial, grd, datos_incertidumbre):
        # Atributos del paciente al llegar, no cambian
        self.id = Paciente.CONTADOR_ID  # Asigna un ID único al Paciente
        self.hospital_llegada = hospital_llegada
        self.requerimiento_inicial = requerimiento_inicial
        self.grd = grd
        self.ti_inicial = datos_incertidumbre["TI"]
        self.camino = deepcopy(datos_incertidumbre["camino"])
        self.espera = deepcopy(datos_incertidumbre["espera"])
        Paciente.CONTADOR_ID += 1  # Incrementa el contador de Pacientes

        # Diccionarios parametros unidades y hospitales inverso (para simplificar el acceso)
        self.dict_hospitales = self.invertir_dict(p.dict_hospitales)
        self.dict_unidades = self.invertir_dict(p.dict_unidades)

        # Atributos que cambian a lo largo de la simulación
        self.log_eventos = []
        self.ti_evento_actual = datos_incertidumbre["TI"]
        self.tiempo_actual = datos_incertidumbre["TI"]
        self.tiempo_ultima_evolucion = 0
        self.hospital_actual = hospital_llegada
        self.unidad_actual = self.unidad_llegada()
        self.unidad_requerida = self.requerimiento_inicial
        self.esperando = True # True si esta esperando y no siendo tratado (revisar en cada ciclo)
        self.costo_social = 0
        self.costos_operativos = 0
        
    # Necesarias para cuando se crean los pacientes
    def invertir_dict(self, diccionario):
        invertido = {v: k for k, v in diccionario.items()}
        return invertido
    
    def unidad_llegada(self):
        if self.hospital_llegada == 0:
            unidad_actual = p.dict_unidades["WL"]
        else:
            unidad_actual = p.dict_unidades["ED"]
        return unidad_actual
    
    # Necesarias durante la simulación
    # Guarda los eventos que le ocurren al paciente en el log_eventos
    def agregar_log_evento(self): # Revisado, funciona bien

        if self.hospital_actual == 0 and self.unidad_actual == p.dict_unidades["WL"]:
            hospital = "WL"
        elif self.unidad_actual == p.dict_unidades["PS"]:
            hospital = "PS"
        else:
            hospital = self.dict_hospitales[self.hospital_actual]

        if self.tiempo_ultima_evolucion == 0:
            tiempo_final = self.tiempo_actual
        elif self.tiempo_ultima_evolucion != 0:
            tiempo_final = self.ti_evento_actual + self.tiempo_ultima_evolucion

        new_row = {
            'ID': self.id,
            'MS_GRD': self.grd,
            'UBICACIÓN': hospital + "_" + self.dict_unidades[self.unidad_actual],
            'TI': self.ti_evento_actual,
            'TF': tiempo_final,
            'HOSPITAL': hospital,
            'UNIDAD': self.dict_unidades[self.unidad_actual]
        }
        self.log_eventos.append(new_row)
    
    # Metodos para calcular los distintos costos en los que incurre el paciente (social y operativo)
    def costo_espera(self, tiempo_espera = 1, agregar = False): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_espera_wl[self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["GA"]: 
            costo = p.dict_costo_espera_ga[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["ED"]: 
            costo = p.dict_costo_espera_ed[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual in (p.dict_unidades["OR"], p.dict_unidades["ICU"], p.dict_unidades["SDU/WARD"]): # (1, 2, 3)
            costo = p.dict_costo_espera_hospitalizado[self.hospital_actual][self.grd][self.unidad_actual][self.unidad_requerida] * tiempo_espera
        else:
            costo = 0
        if agregar:
            self.costo_social += costo
        return costo

    def costo_traslado(self, hospital_destino, agregar = False): # Revisado, funciona bien
        costo = p.dict_costo_traslado[self.hospital_actual][hospital_destino][self.grd][self.requerimiento_inicial]
        if agregar:
            self.costos_operativos += costo
        return costo
    
    def costo_desvio(self, agregar = False): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["ED"]:
            costo = p.dict_costo_derivar_ed[self.hospital_actual][self.grd][self.requerimiento_inicial]
        elif self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_derivar_wl[self.grd][self.requerimiento_inicial]
        if agregar:
            self.costos_operativos += costo
        return costo

    # Metodo para cambio de unidad o PS, creo el log correspondiente antes de cambiar la unidad
    def cambiar_unidad(self, hospital: int, id_unidad: int): # Revisado, funciona bien
        # Agrego el evento al log
        # Esto toma como TF el tiempo actual que es el TI de la unidad destino
        self.agregar_log_evento()

        # Antes de cambiar cualquier valor, veo si hubieron gastos operativos
        if id_unidad == p.dict_unidades["PS"]:
            self.costo_desvio(agregar = True)
        elif self.unidad_actual == p.dict_unidades["ED"] and self.hospital_actual != hospital and id_unidad == p.dict_unidades["ED"]:
            self.costo_traslado(hospital, agregar = True)

        # Actualizo todo lo necesario para el cambio de unidad
        # Actualizo el tiempo de evento actual, osea el TI en la nueva unidad
        self.ti_evento_actual = self.tiempo_actual
        # Actualizo el hospital actual
        self.hospital_actual = hospital
        # Actualizo la unidad actual
        self.unidad_actual = id_unidad
        # Caso de termino de la simulacion
        if self.unidad_actual == p.dict_unidades["PS"]:
            self.agregar_log_evento()
        # Corroborar si el cambio modifico el estado de espera o tratamiento
        self.espera_o_tratamiento()

    # Actualizar el tiempo implica si o si que el paciente estuvo un ciclo mas en la unidad actual
    # Nunca se debera actualizar el tiempo si ya esta en PS o END
    # Nunca se debera actualizar el tiempo si su unidad requerida es END
    def actualizar_tiempo(self): # Revisado, funciona bien
        self.tiempo_actual += 1
        costo_espera = 0
        if self.esperando:
            self.costo_espera(agregar = True)
            costo_espera = self.costo_espera()
        self.espera_o_tratamiento()
        print(f"Paciente {self.id} | Tiempo actual: {self.tiempo_actual} | Costo espera: {costo_espera} | Esperara?: {self.esperando}")
    
    # Ve estado actual, requerimiento y modifica esperando acordemente, tambien modifica unidad_requerida
    def espera_o_tratamiento(self): # Revisado, funciona bien
        if self.unidad_actual  in (p.dict_unidades["OR"], p.dict_unidades["ICU"], p.dict_unidades["SDU/WARD"]):
            if self.unidad_requerida == self.unidad_actual:
                tiempo_siendo_atendido = self.tiempo_actual - self.ti_evento_actual
                if tiempo_siendo_atendido <  self.espera[self.hospital_actual][0]:
                    self.esperando = False
                elif tiempo_siendo_atendido ==  self.espera[self.hospital_actual][0]:
                    self.esperando = True

                    # Elimino la unidad anteriormente requerida y su tiempo de evolucion (este lo guardo)
                    self.camino[self.hospital_actual].pop(0)
                    self.tiempo_ultima_evolucion = self.espera[self.hospital_actual].pop(0)
                    if len(self.camino[self.hospital_actual]) > 0:
                        self.unidad_requerida = self.camino[self.hospital_actual][0]
                    # Si no hay mas unidades en la lista, el paciente es dado de alta, requiere END que es salir
                    else:
                        self.unidad_requerida = p.dict_unidades["END"]
            else:
                self.esperando = True
        elif self.unidad_actual  in (p.dict_unidades["WL"], p.dict_unidades["GA"], p.dict_unidades["ED"]):
            self.esperando = True
        else: # Para PS y END, nunca deberia ser llamada esta funcion desde esas unidades
            pass

    def __str__(self): # Revisado, funciona bien
        if self.hospital_actual == 0 and self.unidad_actual == p.dict_unidades["WL"]:
            hospital = "WL"
        elif self.unidad_actual == p.dict_unidades["PS"]:
            hospital = "PS"
        else:
            hospital = self.dict_hospitales[self.hospital_actual]
        return f"Paciente {self.id} | Requerimiento: {self.dict_unidades[self.unidad_requerida]} | GRD: {self.grd} | {hospital} | Unidad: {self.dict_unidades[self.unidad_actual]} | Social: {self.costo_social} y Operativo: {self.costos_operativos} | Tiempo actual: {self.tiempo_actual}"


In [6]:
class WL:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    al usar deques las operaciones de append y popleft son O(1), lo cual lo hace mas eficiente
    al ser un sistema de colas FIFO"""
    def __init__(self, requerimientos: list, grds: list):
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
    
    # Siempre el primero de cada sublista es el que lleva mas esperando porque llego antes
    def sacar_paciente(self, requerimiento: int, grd: int): # Revisado, funciona bien
        if self.sub_listas[requerimiento][grd]:
            return self.sub_listas[requerimiento][grd].popleft()
        else:
            return None
        
    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada paciente de cada sublista
        for requerimiento in self.sub_listas:
            for grd in self.sub_listas[requerimiento]:
                if self.sub_listas[requerimiento][grd]:
                    for paciente in self.sub_listas[requerimiento][grd]:
                        paciente.actualizar_tiempo()
        self.acumular_costo_ciclo()      
    
    def acumular_costo_ciclo(self): # Revisado, funciona bien
        for requerimiento in self.sub_listas:
            for grd in self.sub_listas[requerimiento]:
                if self.sub_listas[requerimiento][grd]:
                    for paciente in self.sub_listas[requerimiento][grd]:
                        self.costo_acumulado_sub_listas[requerimiento][grd] += paciente.costo_espera()
                else:
                    self.costo_acumulado_sub_listas[requerimiento][grd] += 0

    def __str__(self):
        texto = "Sublistas de WL:"
        for requerimiento in self.sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {len(self.sub_listas[requerimiento][grd])} pacientes"
                if self.sub_listas[requerimiento][grd]:
                    texto += f" | ID Primer paciente: {self.sub_listas[requerimiento][grd][0].id}"
                else:
                    texto += " | Sin pacientes"
        
        texto += "\n\nCostos acumulados de WL:"
        for requerimiento in self.costo_acumulado_sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.costo_acumulado_sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {self.costo_acumulado_sub_listas[requerimiento][grd]} pesos"
        
        return texto

In [7]:
class PS:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    funciona simplemente como un sumidero de pacientes"""
    def __init__(self, requerimientos: list, grds: list):
        self.id_unidad = p.dict_unidades["PS"]
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        self.costo_acumulado_sub_listas[paciente.requerimiento_inicial][paciente.grd] += paciente.costo_desvio()
        paciente.cambiar_unidad(paciente.hospital_actual, self.id_unidad)

    def __str__(self):
        texto = "Sublistas de PS:"
        for requerimiento in self.sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {len(self.sub_listas[requerimiento][grd])} pacientes"
                if self.sub_listas[requerimiento][grd]:
                    texto += f" | {self.sub_listas[requerimiento][grd][0].id}"
                else:
                    texto += " | 0 pacientes"
        texto += "\n\nCostos acumulados de PS:"
        for requerimiento in self.costo_acumulado_sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.costo_acumulado_sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {self.costo_acumulado_sub_listas[requerimiento][grd]} pesos"
                if self.costo_acumulado_sub_listas[requerimiento][grd]:
                    texto += f" | {self.sub_listas[requerimiento][grd][0].id}"
                else:
                    texto += " | 0 pacientes"
        return texto

In [8]:
class END:
    """
    Funciona con varias sublistas, una por cada combinación de requerimiento y grd,
    almacena pacientes terminados.
    """
    def __init__(self, requerimientos: list, grds: list):
        self.id_unidad = p.dict_unidades["END"]
        self.sub_listas = {}
        self.crear_sublistas(requerimientos, grds)

    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()

    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        """
        Agrega un paciente a la sublista correspondiente según su requerimiento_inicial y grd.
        """
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        paciente.cambiar_unidad(paciente.hospital_actual, self.id_unidad)

    def __str__(self):
        lines = ["Sublistas de Termino:"]
        for req, dict_grd in self.sub_listas.items():
            for grd, dq in dict_grd.items():
                lines.append(f"  - Requerimiento {req} GRD {grd}: {len(dq)} pacientes")
        return "\n".join(lines)

In [ ]:
class Unidad:
    """Clase para las unidades, se usan listas para almacenar pacientes debido a su pequeño tamaño, 
    en el peor de los casos el SDU/WARD tiene 165 pacientes, por lo que una lista es suficiente y
    es facil de iterar sobre ella. Todas las unidades de los hospitales heredan de la clase Unidad, 
    se comportan exactamente igual solo varia el id de la unidad y la capacidad, que se obtiene del 
    diccionario de capacidades"""
    def __init__(self, id_unidad, hospital, capacidad):
        self.id_unidad = id_unidad
        self.hospital = hospital
        self.capacidad = capacidad
        self.pacientes = []

    @property # Revisado, funciona bien
    def ocupacion(self):
        return len(self.pacientes)
    
    @property # Revisado, funciona bien
    def ocupacion_porcentual(self):
        return len(self.pacientes)/self.capacidad
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        # Si el paciente es None, no se agrega, osea que no estaba donde se queria sacar
        if paciente:
            if self.ocupacion < self.capacidad and paciente not in self.pacientes:
                # Si el paciente ya esta en la lista, no se agrega
                # Si la ocupacion es menor a la capacidad, se agrega el paciente
                self.pacientes.append(paciente)
                # Al agregar el paciente, se actualiza su unidad actual con el metodo de la clase
                paciente.cambiar_unidad(self.hospital, self.id_unidad)
                return True
            else:
                return False
        else:
            return False
        
    def sacar_paciente(self, paciente): # Revisado, funciona bien
        if self.ocupacion > 0 and paciente in self.pacientes:
            # Si el paciente no esta en la lista, no se saca
            # Si la ocupacion es mayor a 0, se saca el paciente
            # Cada instancia de paciente es unica, por lo que no hay problema al usar remove
            self.pacientes.remove(paciente) 
            return paciente
        else:
            return None
    
    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada paciente de la unidad
        for paciente in self.pacientes:
            paciente.actualizar_tiempo()
    
    def __str__(self):
        # Diccionario invertido para obtener el nombre de la unidad
        unit_names = {
            p.dict_unidades["OR"]: "OR",
            p.dict_unidades["ICU"]: "ICU",
            p.dict_unidades["SDU/WARD"]: "SDU/WARD",
            p.dict_unidades["GA"]: "GA",
            p.dict_unidades["ED"]: "ED"
        }
        unit_name = unit_names.get(self.id_unidad, f"Unidad {self.id_unidad}")
        description = f"{unit_name} (Hospital {self.hospital}): {self.ocupacion}/{self.capacidad} pacientes"
        if self.pacientes:
            description += f"\nPacientes: {len(self.pacientes)}, {self.ocupacion_porcentual:.2%} ocupación"
            if len(self.pacientes) <= 20:  # Only show details for a small number of patients
                for i, paciente in enumerate(self.pacientes):
                    description += f"\n  {i+1}. Paciente ID {paciente.id}, GRD {paciente.grd}"
        return description

class Or(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["OR"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Icu(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["ICU"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class SduWard(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["SDU/WARD"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Ga(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["GA"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Ed(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["ED"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)
    
    # Se redefine el metodo para que no considere la ocupacion de la unidad, solo para ED
    def agregar_paciente(self, paciente: Paciente): # Revisar
        # Si el paciente es None, no se agrega, osea que no estaba donde se queria sacar
        if paciente:
            if paciente not in self.pacientes:
                # Si el paciente ya esta en la lista, no se agrega
                # Si la ocupacion es menor a la capacidad, se agrega el paciente
                self.pacientes.append(paciente)
                # Al agregar el paciente, se actualiza su unidad actual con el metodo de la clase
                paciente.cambiar_unidad(self.hospital, self.id_unidad)
                return True
            else:
                return False
        else:
            return False

In [10]:
class Hospital:
    """Representa un hospital con una instancia de cada unidad disponible."""
    def __init__(self, hospital_id):
        self.hospital = hospital_id
        # Instanciar una unidad de cada tipo
        self.OR = Or(hospital_id)
        self.ICU = Icu(hospital_id)
        self.SDU_WARD = SduWard(hospital_id)
        self.GA = Ga(hospital_id)
        self.ED = Ed(hospital_id)
        # Diccionario para acceso y recorridos
        self.unidades = {
            p.dict_unidades["OR"]: self.OR,
            p.dict_unidades["ICU"]: self.ICU,
            p.dict_unidades["SDU/WARD"]: self.SDU_WARD,
            p.dict_unidades["GA"]: self.GA,
            p.dict_unidades["ED"]: self.ED,
        }

    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada unidad del hospital
        for unidad in self.unidades.values():
            unidad.actualizar_tiempo()
    
    def ocupacion(self): # Revisar
        # Devuelve la ocupación de todo el hospital
        ocupacion = {
            unidad_id: {"ocupacion": unidad.ocupacion, "capacidad": unidad.capacidad, "porcentual": unidad.ocupacion_porcentual,}
            for unidad_id, unidad in self.unidades.items()
        }
        return ocupacion
    
    
    def agregar_paciente(self, paciente: Paciente, unidad: int): # Revisado, funciona bien
        # Agrega un paciente a la unidad correspondiente.
        if unidad in self.unidades:
            return self.unidades[unidad].agregar_paciente(paciente)
        else:
            raise ValueError(f"Unidad {unidad} no válida en el hospital {self.hospital}.")
    
    def sacar_paciente(self, paciente: Paciente, unidad: int): # Revisado, funciona bien
        # Saca un paciente de la unidad correspondiente.
        if unidad in self.unidades:
            return self.unidades[unidad].sacar_paciente(paciente)
        else:
            raise ValueError(f"Unidad {unidad} no válida en el hospital {self.hospital}.")
    
    def __str__(self):
        lines = [f"Hospital {self.hospital}:"]
        for unidad_id, unidad in self.unidades.items():
            lines.append(f"  - {unidad_id}: {unidad.ocupacion}/{unidad.capacidad}")
        return "\n".join(lines)

Reunion con profesor, datos utiles:
- Todo paciente debe ser aceptado sin excepción
- Si se tiene que abusar del presupuesto, se abusa para que exista factibilidad, esta factibilidad se refiere a atender a todos los pacientes y no sobrepasar ninguna ocupación (especialmente en las ED o los GA)
- En situaciones cuando todo se satura la simulacion base no hace sentido porque el modelo cambia, por eso no tomar en cuenta esos comportamientos extraños
- Si llegan mas de 15 a una ED determinada, da lo mismo, lo que si importa es que cuando termine de tomar decisiones en ese periodo, no hayan mas de 15, NUNCA pueden haber mas de 15 y que se actualice el tiempo
- Si hay mas de 15 y no quedan camas, o los traslado o los mando al PS, no los 2 porque seria un malgasto
- En situaciones normales donde existe factibilidad no se sobrepasa el presupuesto por ningun motivo
- Si el quiebre de budget es inminente se activa modo "Quiebre Budget" que se comportara distinto
- WL tiene capacidad maxima (calcular) y tambien tiene tiempo maximo por paciente (400 ciclos exactamente)
- Actualmente el GA en el caso base no tiene tiempo maximo por paciente (por lo que es una mejora facil para la nueva politica)

Modelo Miope Decisiones:
1. Dar de alta a todos los pacientes de todos los SDU_WARD.
2. Por cada hospital crear listas de prioridades con pacientes de su ED, GA y (OR, ICU y SDU_WARD).
3. Estas listas se tienen que crear de la siguiente manera (esta estapa esta dentro de un ciclo).
    1. A estas listas le modifico el costo a los pacientes que provengan de unidades de atención del hospital, esto se debe a que el costo asociado de estos pacientes es su costo de esperar en la unidad actual mas el costo de la primera persona con prioridad que usaria su cama en caso de ser liberada, es decir estos pacientes pueden tener una cola detras de ellos.
    2. Con las listas modificadas agrego a estos paciente a la unidad OR en orden, paro si se acaban las camas o se acaba la lista.
    3. Luego paso a ICU, vuelvo a modificar las listas porque la persona que venia a la cola de otra podria haber sido hospitalizada, por lo que ahora vuelvo a ver quien esta primero a la cola, agrego pacientes a la unidad, paro si se acaban las camas o se acaba la lista.
    4. Realizo el mismo procedimiento para el SDU_WARD.
    5. Este loop se detiene cuando en dos iteraciones seguidas las camas libres son las mismas tanto para OR, ICU y SDU_WARD. Esto significa que se realizaron todos los movimientos posibles.
4. Ahora a nivel ED de los hospitales veo si quedaron pacientes sin atender, por que se lleno su hospital.
5. Si es que existen pacientes esperando en ED y cupos para ellos en otro hospital, los traslado.
6. Si todavia siguen esperando pacientes en alguna ED los derivo al sistema privado.
7. Con los ED vacios (en la gran gran gran mayoria de los casos) puedo pasar al siguiente paso.
8. Si ya nadie mas quiere entrar a las unidades desde ED, GA y (OR, ICU y SDU_WARD) puedo meter a gente directamente desde lista de espera
9. En estos casos esa gente no va a esperar en GA, nunca conviene que esperen en GA.
10. Una vez realizado todo lo mencionado anteriormente, se terminarian las decisiones para ese determinado ciclo.

11. Consideración, casos extraños:
    1. Cada 400 ciclos de espera en WL los pacientes seran expulsados de esta, de mi depende meterlos a GA o mandarlos directamente a PS, en ese orden.
    2. Cuando se llena la WL (capacidad maxima: ) los pacientes seran expulsados de esta, de mi depende meterlos a GA o mandarlos directamente a PS, en ese orden.
    3. Cuando el costo operativo supera el budget y no hay otra opcion, gasto mas de este, de manera eficiente hasta asegurar factibilidad.

In [ ]:
# Ojo recordar que los pacientes en WL se sacan en el ciclo 4800/12=400, maximo 400 ciclos y los atiendo o mando a PS, uno de dos
# El budget puede ser sobrepasado solo si es que no hay otras opciones y si o si debo mover pacientes
class Modelo:

    def __init__(self):
        self.decisiones = []
        self.camas_libres = {
            p.dict_hospitales["Hospital_1"]: {
                p.dict_unidades["OR"]: 0,
                p.dict_unidades["ICU"]: 0,
                p.dict_unidades["SDU/WARD"]: 0
            },
            p.dict_hospitales["Hospital_2"]: {
                p.dict_unidades["OR"]: 0,
                p.dict_unidades["ICU"]: 0,
                p.dict_unidades["SDU/WARD"]: 0
            },
            p.dict_hospitales["Hospital_3"]: {
                p.dict_unidades["OR"]: 0,
                p.dict_unidades["ICU"]: 0,
                p.dict_unidades["SDU/WARD"]: 0
            }
        }

    def evolucion_hospitalizados(self, simulacion): # Revisar
        decisiones_hospitalizados = {
            "pacientes_de_alta": [],
            "pacientes_cambio_a_camas_libres": [],
            "pacientes_cambios_simultaneos": []
        }

        for id_hospital, hospital in simulacion.hospitales.items():
            if id_hospital != 0: # No considero WL
                ocupacion_hospital = hospital.ocupacion()
                
                # Primero reviso que pacientes terminan su evolucion en el SDU_WARD
                pacientes_de_alta = [] # Revisar

                for paciente in hospital.SDU_WARD.pacientes:
                    if paciente.unidad_requerida == p.dict_unidades["END"]:
                        pacientes_de_alta.append(paciente)
                
                pacientes_cambio_a_camas_libres = [] # Revisado, funciona bien
                
                # Ahora cuento con las ocupaciones reales de cada unidad
                ocupacion_or = ocupacion_hospital[p.dict_unidades["OR"]]["ocupacion"]
                capacidad_or = ocupacion_hospital[p.dict_unidades["OR"]]["capacidad"]
                ocupacion_icu = ocupacion_hospital[p.dict_unidades["ICU"]]["ocupacion"]
                capacidad_icu = ocupacion_hospital[p.dict_unidades["ICU"]]["capacidad"]
                ocupacion_sdu_ward = ocupacion_hospital[p.dict_unidades["SDU/WARD"]]["ocupacion"] - len(pacientes_de_alta)
                capacidad_sdu_ward = ocupacion_hospital[p.dict_unidades["SDU/WARD"]]["capacidad"]
                
                # Reviso si hay pacientes esperando a ser cambiados de unidad en cada unidad
                esperando_en_or = []
                esperando_en_icu = []
                esperando_en_sdu_ward = []
                requieren_or = []
                requieren_icu = []
                requieren_sdu_ward = []

                for paciente in hospital.OR.pacientes:
                    if paciente.esperando:
                        esperando_en_or.append(paciente)
                        if paciente.unidad_requerida == p.dict_unidades["ICU"]:
                            requieren_icu.append(paciente)
                        elif paciente.unidad_requerida == p.dict_unidades["SDU/WARD"]:
                            requieren_sdu_ward.append(paciente)

                for paciente in hospital.ICU.pacientes:
                    if paciente.esperando:
                        esperando_en_icu.append(paciente)
                        if paciente.unidad_requerida == p.dict_unidades["OR"]:
                            requieren_or.append(paciente)
                        elif paciente.unidad_requerida == p.dict_unidades["SDU/WARD"]:
                            requieren_sdu_ward.append(paciente)

                for paciente in hospital.SDU_WARD.pacientes:
                    if paciente.esperando:
                        # No considero los que estan dados de alta
                        if paciente.unidad_requerida != p.dict_unidades["END"]:
                            esperando_en_sdu_ward.append(paciente)
                            if paciente.unidad_requerida == p.dict_unidades["OR"]:
                                requieren_or.append(paciente)
                            elif paciente.unidad_requerida == p.dict_unidades["ICU"]:
                                requieren_icu.append(paciente)
                
                # Ordeno las listas de menor costo de espera a mayor
                esperando_en_or.sort(key=lambda x: x.costo_espera())
                esperando_en_icu.sort(key=lambda x: x.costo_espera())
                esperando_en_sdu_ward.sort(key=lambda x: x.costo_espera())
                requieren_or.sort(key=lambda x: x.costo_espera())
                requieren_icu.sort(key=lambda x: x.costo_espera())
                requieren_sdu_ward.sort(key=lambda x: x.costo_espera())

                # Ahora se revisa la disponibilidad de camas
                camas_libres_or = capacidad_or - ocupacion_or
                camas_libres_icu = capacidad_icu - ocupacion_icu
                camas_libres_sdu_ward = capacidad_sdu_ward - ocupacion_sdu_ward

                camas_libres_siguen_cambiando = True

                while camas_libres_siguen_cambiando:

                    camas_libres_or_anterior = camas_libres_or 
                    camas_libres_icu_anterior = camas_libres_icu
                    camas_libres_sdu_ward_anterior = camas_libres_sdu_ward
                    
                    contador = 0
                    for i in range(camas_libres_sdu_ward):
                        if len(requieren_sdu_ward) > 0:
                            paciente = requieren_sdu_ward.pop()
                            pacientes_cambio_a_camas_libres.append(paciente)
                            contador += 1
                            if paciente in esperando_en_or:
                                esperando_en_or.remove(paciente)
                                camas_libres_or += 1
                            elif paciente in esperando_en_icu:
                                esperando_en_icu.remove(paciente)
                                camas_libres_icu += 1
                    camas_libres_sdu_ward -= contador
                    
                    contador = 0
                    for i in range(camas_libres_icu):
                        if len(requieren_icu) > 0:
                            paciente = requieren_icu.pop()
                            pacientes_cambio_a_camas_libres.append(paciente)
                            contador += 1
                            if paciente in esperando_en_or:
                                esperando_en_or.remove(paciente)
                                camas_libres_or += 1
                            elif paciente in esperando_en_sdu_ward:
                                esperando_en_sdu_ward.remove(paciente)
                                camas_libres_sdu_ward += 1
                    camas_libres_icu -= contador

                    contador = 0
                    for i in range(camas_libres_or):
                        if len(requieren_or) > 0:
                            paciente = requieren_or.pop()
                            pacientes_cambio_a_camas_libres.append(paciente)
                            contador += 1
                            if paciente in esperando_en_icu:
                                esperando_en_icu.remove(paciente)
                                camas_libres_icu += 1
                            elif paciente in esperando_en_sdu_ward:
                                esperando_en_sdu_ward.remove(paciente)
                                camas_libres_sdu_ward += 1
                    camas_libres_or -= contador

                    if camas_libres_or == camas_libres_or_anterior and camas_libres_icu == camas_libres_icu_anterior and camas_libres_sdu_ward == camas_libres_sdu_ward_anterior:
                        camas_libres_siguen_cambiando = False
                # Ahora se revisa si hay pacientes esperando en las unidades que no son OR, ICU o SDU/WARD

                """En caso de llenarse las unidades puede darse que pacientes de distintas unidades
                intercambien camas entre si de manera simultanea, las unidades seguirian llenas
                pero tratando efectivamente a los pacientes y no teniendolos en espera, para eso
                necesito listas mas detalladas"""

                pacientes_cambios_simultaneos = [] # Revisado, funciona bien

                or_icu = []
                icu_or = []
                icu_sdu_ward = []
                sdu_ward_icu = []
                sdu_ward_or = []
                or_sdu_ward = []

                for paciente in esperando_en_or:
                    if paciente.unidad_requerida == p.dict_unidades["ICU"]:
                        or_icu.append(paciente)
                    elif paciente.unidad_requerida == p.dict_unidades["SDU/WARD"]:
                        or_sdu_ward.append(paciente)
                
                for paciente in esperando_en_icu:
                    if paciente.unidad_requerida == p.dict_unidades["OR"]:
                        icu_or.append(paciente)
                    elif paciente.unidad_requerida == p.dict_unidades["SDU/WARD"]:
                        icu_sdu_ward.append(paciente)

                for paciente in esperando_en_sdu_ward:
                    if paciente.unidad_requerida == p.dict_unidades["OR"]:
                        sdu_ward_or.append(paciente)
                    elif paciente.unidad_requerida == p.dict_unidades["ICU"]:
                        sdu_ward_icu.append(paciente)

                # Ordeno las listas de menor costo de espera a mayor
                or_icu.sort(key=lambda x: x.costo_espera())
                icu_or.sort(key=lambda x: x.costo_espera())
                icu_sdu_ward.sort(key=lambda x: x.costo_espera())
                sdu_ward_icu.sort(key=lambda x: x.costo_espera())
                sdu_ward_or.sort(key=lambda x: x.costo_espera())
                or_sdu_ward.sort(key=lambda x: x.costo_espera())

                cambios_entre_or_icu = min(len(or_icu), len(icu_or))
                cambios_entre_or_sdu_ward = min(len(or_sdu_ward), len(sdu_ward_or))
                cambios_entre_icu_sdu_ward = min(len(icu_sdu_ward), len(sdu_ward_icu))

                for i in range(cambios_entre_or_icu):
                    paciente = or_icu.pop()
                    pacientes_cambios_simultaneos.append(paciente)
                    paciente = icu_or.pop()
                    pacientes_cambios_simultaneos.append(paciente)
                
                for i in range(cambios_entre_or_sdu_ward):
                    paciente = sdu_ward_or.pop()
                    pacientes_cambios_simultaneos.append(paciente)
                    paciente = or_sdu_ward.pop()
                    pacientes_cambios_simultaneos.append(paciente)
                
                for i in range(cambios_entre_icu_sdu_ward):
                    paciente = icu_sdu_ward.pop()
                    pacientes_cambios_simultaneos.append(paciente)
                    paciente = sdu_ward_icu.pop()
                    pacientes_cambios_simultaneos.append(paciente)

                for paciente in pacientes_de_alta:
                    decisiones_hospitalizados["pacientes_de_alta"].append(paciente)
                for paciente in pacientes_cambio_a_camas_libres:
                    decisiones_hospitalizados["pacientes_cambio_a_camas_libres"].append(paciente)
                for paciente in pacientes_cambios_simultaneos:
                    decisiones_hospitalizados["pacientes_cambios_simultaneos"].append(paciente)

        for etapa in ["pacientes_de_alta", "pacientes_cambio_a_camas_libres", "pacientes_cambios_simultaneos"]: # Revisar    
            dict_temporal = {}
            for paciente in decisiones_hospitalizados[etapa]:
                dict_temporal[paciente] = {"hospital": paciente.hospital_actual, "unidad": paciente.unidad_requerida}
            self.decisiones.append(dict_temporal)
        
        return 
                
    def tomar_decisiones(self, simulacion):
        # Primero realizo los cambios de unidad de los pacientes ya hospitalizados
        self.evolucion_hospitalizados(simulacion)

        
        return self.decisiones

In [12]:
class Simulacion:

    def __init__(self, T_max, seed, ciclos):
        """Se empieza generando los pacientes con la semilla de random
        y la cantidad de ciclos que se desean crear pacientes"""
        generar_pacientes(seed, ciclos)
        self.T_max = T_max
        self.pacientes_separados_por_llegada = self.cargar_pacientes_separados_por_llegada()
        print("Pacientes separados por llegada cargados")
        self.ciclos = ciclos
        self.modelo = Modelo()
        self.hospital_1 = Hospital(1)
        self.hospital_2 = Hospital(2)
        self.hospital_3 = Hospital(3)
        self.wl = WL([1, 2, 3], [5, 6, 7, 8])
        self.hospitales = { # Para acceder a los hospitales por su id
            0: self.wl,
            p.dict_hospitales["Hospital_1"]: self.hospital_1,
            p.dict_hospitales["Hospital_2"]: self.hospital_2,
            p.dict_hospitales["Hospital_3"]: self.hospital_3
        }
        self.ps = PS([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
        self.end = END([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
        self.unidades_termino = { # Para acceder a las unidades de termino por su id
            p.dict_unidades["PS"]: self.ps,
            p.dict_unidades["END"]: self.end
        }
        self.budget = p.budget
        self.tasa_descuento = p.tasa_descuento
        print("Clase Simulacion instanciada")

    # Funcion necesaria al momento de instanciar la clase
    def cargar_pacientes_separados_por_llegada(self): # Revisado, funciona bien
        # Cargo los datos necesarios y transformo las llaves nuevamente a int (que se habian vuelto str)
        incertidumbre = {}
        for hospital in range(0,4): # 0 son llegadas a WL
            incertidumbre[hospital] = {}
            for requerimiento in range(1,4):
                incertidumbre[hospital][requerimiento] = {}

        with open("resultados incertidumbre/incertidumbre_simulada.json", "r") as file:
            incertidumbre_keys_str = json.load(file)

        for hospital in range(0,4): # 0 son llegadas a WL
            for requerimiento in range(1,4):
                for grd in range(1,9):
                    incertidumbre[hospital][requerimiento][grd] = []
                    lista = incertidumbre_keys_str[str(hospital)][str(requerimiento)][str(grd)]
                    if lista != []:
                        for data in lista:
                            arreglado = {
                            'TI': data["TI"],
                            'camino': {1: data['camino']["1"], 2: data['camino']["2"], 3: data['camino']["3"]},
                            'espera': {1: data['espera']["1"], 2: data['espera']["2"], 3: data['espera']["3"]}
                            }
                            incertidumbre[hospital][requerimiento][grd].append(arreglado)
                    else:
                        incertidumbre[hospital][requerimiento][grd] = []

        # Se instancian a todos los pacientes a partir de los datos de incertidumbre
        pacientes = {}
        lista_pacientes = []
        # 0 son llegadas a WL
        for hospital in range(0,4): 
            pacientes[hospital] = {}
            for requerimiento in range(1,4):
                pacientes[hospital][requerimiento] = {}
                for grd in range(1,9):
                    pacientes[hospital][requerimiento][grd] = []
                    cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
                    if cantidad_pacientes != 0:
                        for i in range(cantidad_pacientes):
                            paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                            pacientes[hospital][requerimiento][grd].append(paciente)
                            lista_pacientes.append(paciente)
                    else:
                        pacientes[hospital][requerimiento][grd] = []

        # Se generan tantas listas como dias con llegadas haya
        pacientes_separados_por_llegada = {}
        for paciente in lista_pacientes:
            ciclo = paciente.ti_inicial
            if ciclo not in pacientes_separados_por_llegada:
                pacientes_separados_por_llegada[ciclo] = []
            pacientes_separados_por_llegada[ciclo].append(paciente)

        return pacientes_separados_por_llegada
    
    # Funciones necesarias al momento de simular
    def agregar_pacientes_ciclo_a_wl(self, ciclo): # Revisado, funciona bien
        pacientes_ciclo = self.pacientes_separados_por_llegada.get(ciclo, [])
        for paciente in pacientes_ciclo:
            # hospital 0 es WL
            if paciente.hospital_llegada == 0:
                self.wl.agregar_paciente(paciente)
            else:
                pass

    def agregar_pacientes_ciclo_a_ed(self, ciclo): # Revisar bien tema de la capacidad de las ED, puede arrojar errores
        pacientes_ciclo = self.pacientes_separados_por_llegada.get(ciclo, [])
        for paciente in pacientes_ciclo:
            # si hospital es 1, 2 o 3 llegan a ED
            if paciente.hospital_llegada != 0:
                self.hospitales[paciente.hospital_llegada].agregar_paciente(paciente, paciente.unidad_actual)
            else:
                pass

    def sacar_paciente(self, paciente): # Revisado, funciona bien
        # Retorna el paciente que se saca de la unidad, o None si no se pudo sacar
        if paciente.hospital_actual == 0:
            return self.hospitales[paciente.hospital_actual].sacar_paciente(paciente.unidad_requerida, paciente.grd)  
        else:
            return self.hospitales[paciente.hospital_actual].sacar_paciente(paciente, paciente.unidad_actual)

    def agregar_paciente(self, paciente, hospital, unidad): # Revisado, funciona bien
        # Retorna True si se agrega el paciente, False si no
        if unidad not in (p.dict_unidades["PS"], p.dict_unidades["END"]):
            return self.hospitales[hospital].agregar_paciente(paciente, unidad)
        else:
            # No retornan nada porque siempre se agrega el paciente
            self.unidades_termino[unidad].agregar_paciente(paciente)
            return True

    def implementar_decisiones(self, decisiones: list): # Revisar
        """Las decisiones son una lista de diccionarios, cada uno con la siguiente estructura:
        {
            paciente: {"hospital": hospital, "unidad": unidad},
            ...
            paciente: {"hospital": hospital, "unidad": unidad},
        }
        Estas se deben implementar en el orden de la lista, se sacan a todos los pacientes de su
        unidad actual y se cambian a la unidad y hospital que se indica en el diccionario. Luego se 
        pasa al siguiente diccionario y se repite el proceso. Se hace de esta manera porque a veces
        puede ocurrir que dos pacientes tengan que ser cambiados entre si, por lo que los cambios
        deben ser simultaneos por cada diccionario, para sacar a un paciente no necesito saber donde
        esta ya que cada paciente contiene su unidad y hospital actual.
        """
        for decision in decisiones:
            # Saco a todos los pacientes de su unidad actual
            for paciente in decision:
                sacado = self.sacar_paciente(paciente)
                # Esto solo para pruebas, despues se elimina
                if sacado is paciente:
                    print(f"Paciente {paciente.id} sacado exitosamente")
                else:
                    print(f"Paciente {paciente.id} no se pudo sacar")
            # Agrego a todos los pacientes a su nueva unidad
            for paciente, destino in decision.items():
                if self.agregar_paciente(paciente, destino["hospital"], destino["unidad"]):
                    print(f"Paciente {paciente.id} agregado exitosamente a {destino['hospital']} en {destino['unidad']}")

    def actualizar_tiempo(self): # Revisado, funciona bien
        self.hospital_1.actualizar_tiempo()
        self.hospital_2.actualizar_tiempo()
        self.hospital_3.actualizar_tiempo()
        self.wl.actualizar_tiempo()
        # Incrementar el tiempo del sistema
        self.T += 1

    def entregar_log_pacientes_terminados_como_data_frame(self): # Revisado, funciona bien
        log_completo = []
        for requerimiento in [1, 2, 3]:
            for grd in [1, 2, 3, 4, 5, 6, 7, 8]:
                for paciente in self.end.sub_listas[requerimiento][grd]:
                    for evento in paciente.log_eventos:
                        log_completo.append(evento)
                for paciente in self.ps.sub_listas[requerimiento][grd]:
                    for evento in paciente.log_eventos:
                        log_completo.append(evento)
        # Convertir la lista de eventos a un DataFrame
        df_log = pd.DataFrame(log_completo)
        # Ordenar el DataFrame por ID y TI y TF
        df_log.sort_values(by=['ID', 'TI', 'TF'], inplace=True)
        # Resetear el índice
        df_log.reset_index(drop=True, inplace=True)
        return df_log


    def simular(self):
        self.T = 1  # Inicializa el tiempo del sistema en 1

        # Bucle principal de simulación
        while self.T <= self.T_max:

            # Agregar pacientes a WL según el ciclo actual
            self.agregar_pacientes_ciclo_a_wl(self.T)

            # Agregar pacientes a las ED según el ciclo actual
            self.agregar_pacientes_ciclo_a_ed(self.T)

            # Entrego el estado de la simulación al modelo para tomar decisiones
            decisiones = self.modelo.tomar_decisiones(self) # La clase Simulacion se pasa como argumento

            # Se implementan las decisiones del modelo
            self.implementar_decisiones(decisiones)

            # Actualizar el tiempo de cada unidad y paciente
            self.actualizar_tiempo()
            
        
        # Al finalizar la simulación, se entregan los logs de los pacientes terminados
        return self.entregar_log_pacientes_terminados_como_data_frame()
        
    def __str__(self):
        pass

In [13]:
simu = Simulacion(5001, 42, 5000)

La incertidumbre de los pacientes ha sido generada y guardada en el archivo JSON.
Pacientes separados por llegada cargados
Clase Simulacion instanciada
